In [1]:
import aoc

In [2]:
data = aoc.read("day5.txt")
seeds = aoc.to_ints(data[0].split(":")[1].split())
levels = data[1:]

In [3]:
def create_translation_table(maps_):
    dct = {}
    for dest_range_start, src_range_start, range_length in maps_:
        # It's important to use ranges as keys, so that the dictionaries are not gigantic
        # Interestingly, this closely relates to one of the most upvoted python questions on SO
        # https://stackoverflow.com/questions/30081275/why-is-1000000000000000-in-range1000000000000001-so-fast-in-python-3
        dct[range(src_range_start, src_range_start + range_length)] = (
            dest_range_start - src_range_start
        )
    return dct


def parse_level(level):
    level = level.split("\n")[1:]
    level = [aoc.to_ints(map_.split()) for map_ in level]
    return create_translation_table(level)


parsed_levels = [parse_level(level) for level in levels]

In [ ]:
def handle_loc(loc, level):
    for rng, add in level.items():
        if loc in rng:
            return loc + add

    return loc


current_location = seeds.copy()
for level in parsed_levels:
    current_location = [handle_loc(l, level) for l in current_location]
min(current_location)

# Part 2
These numbers are so huge, we should probably calculate it

In [ ]:
def find_overlapping_and_non_overlapping(this_range, rng_object):
    my_start, my_end = this_range

    overlapping_ranges = []
    nonoverlapping_ranges = []
    if my_start >= rng_object.stop or my_end <= rng_object.start:
        nonoverlapping_ranges.append(this_range)
    elif my_start >= rng_object.start and my_end <= rng_object.stop:
        overlapping_ranges.append(this_range)
    elif rng_object.start >= my_start and rng_object.stop < my_end:
        nonoverlapping_ranges.append((my_start, rng_object.start))
        nonoverlapping_ranges.append((rng_object.stop, my_end))
        overlapping_ranges.append((rng_object.start, rng_object.stop))
    elif rng_object.start >= my_start and rng_object.stop >= my_end:
        nonoverlapping_ranges.append((my_start, rng_object.start))
        overlapping_ranges.append((rng_object.start, my_end))
    elif rng_object.start < my_start and rng_object.stop < my_end:
        overlapping_ranges.append((my_start, rng_object.stop))
        nonoverlapping_ranges.append((rng_object.stop, my_end))
    else:
        raise ValueError(f"Unknown situation: {this_range=}, {rng_object=}")

    return overlapping_ranges, nonoverlapping_ranges


def handle_loc(loc, level):
    for rng_object, add in level.items():
        overlapping_ranges, nonoverlapping_ranges = (
            find_overlapping_and_non_overlapping(loc, rng_object)
        )
        if overlapping_ranges:
            new_range = tuple(x + add for x in overlapping_ranges[0])
            return new_range, nonoverlapping_ranges

    return False, nonoverlapping_ranges


def handle_level(locs, level):
    new_locs = []
    current_locs = locs.copy()
    while current_locs:
        loc = current_locs.pop()
        overlapping_ranges, nonoverlapping_ranges = handle_loc(loc, level)
        if overlapping_ranges:
            # It is possible the non-overlapping part of this mapping still overlaps with another map
            current_locs.extend(nonoverlapping_ranges)
            new_locs.append(overlapping_ranges)
        else:
            new_locs.extend(nonoverlapping_ranges)
    return new_locs


# end is exclusive to be consistent with range objects
new_seed_locations = [
    (start, start + length + 1) for start, length in zip(seeds[::2], seeds[1::2])
]
current_location = new_seed_locations.copy()
for level in parsed_levels:
    current_location = handle_level(current_location, level)

# Since we're looking for the first place and start is always before end we only check that part
print(min(start for start, end in current_location))